In [12]:
import types
from __future__ import annotations

In [33]:
# so first, have to generate new scalars from old scalars
# first implement an add function

class Node:

  def __init__(self, value: float) -> None:
    self.value = value
    self.grad = None
  
  def __add__(self, other: "Node") -> "Node":
    return Node(self.value + other.value)
  
  def __sub__(self, other: "Node") -> "Node":
    return Node(self.value - other.value)
  
  def __mul__(self, other: "Node") -> "Node":
    return Node(self.value * other.value)
  
  def backward(self) -> None:
    self.grad = 1
    # i think you do need a topological sort
    # if there are nodes that are both x away from the loss node, but they have dependencies between them, this could be problematic

  def __repr__(self) -> str:
    return f"Scalar(value={self.value})"

In [37]:
a = Node(3)
b = Node(2)
c = Node(-1)
d = Node(4)
e = (a * b + c) * d
l = e + e
l

Scalar(value=40)